<a href="https://colab.research.google.com/github/gabibu/nlp/blob/master/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


*As a preparation for this task, download the data files from the course git repository.

The relevant files are under **lm-languages-data-new**:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)
*   test.csv (or the equivalent JSON file)





In [7]:
!git clone https://github.com/kfirbar/nlp-course.git

Cloning into 'nlp-course'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 53 (delta 23), reused 32 (delta 8), pack-reused 0
Unpacking objects: 100% (53/53), done.




---



**Important note: please use only the files under lm-languages-data-new and NOT under lm-languages-data**


---



In [8]:

!ls nlp-course/lm-languages-data-new


en.csv	 es.json  in.csv   it.json  pt.csv    test.json   tl.csv
en.json  fr.csv   in.json  nl.csv   pt.json   tests.csv   tl.json
es.csv	 fr.json  it.csv   nl.json  test.csv  tests.json


**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [10]:
import os
import io
import pathlib
import pandas as pd
import json 

folder = 'nlp-course/lm-languages-data-new'

def read_tweets(file_path):
  file_suffix = pathlib.Path(file_path).suffix
  if file_suffix == '.json':
    with open(file_path) as f:
      data = json.load(f)
      return [tweet.lower() for tweet in data['tweet_text'].values()]
      
        
  elif file_suffix == '.csv':
    df = pd.read_csv(file_path, header=0)
    return [tweet.lower() for tweet in df['tweet_text'].tolist()]

def preprocess():
  vocab = set()
  for file_name in os.listdir(folder):
    file_path = os.path.join(folder, file_name)
    tweets = read_tweets(file_path)

    vocab.update({word for tweet in tweets for word in tweet.lower()})

  return vocab
        

**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [11]:
from collections import defaultdict
import numpy as np 

start = '<S>'
end = '<E>'

def build_probs_model(n, vocabulary, tweets, add_one):
    counts = {}

    num_of_tokens = 0.0
    for tweet in tweets:
        characters = list(tweet)

        tokens = [start] + characters + [end] if n > 1 else characters

        num_of_tokens += len(tokens)

        if len(tokens) < n:
            continue

        for i in range(n, len(tokens)):
            current_tokens = tokens[i - n: i]

            character = current_tokens[-1]

            if n > 1:
                n_key = ''.join(current_tokens[0: -1])

                if n_key not in counts:
                    counts[n_key] = {}
                    
                if character not in counts[n_key]:
                    counts[n_key][character] = 0.0

                counts[n_key][character] += 1.0
            else:
                if character not in counts:
                    counts[character] = 0.0

                counts[character] += 1


    if n == 1:
        
        if add_one:
            model = defaultdict(lambda : 1.0/len(vocabulary), {word: (count + 1.0) / (num_of_tokens + len(vocabulary)) for (word, count) in counts.items()})
        else:
          model = defaultdict(lambda : 0.0, {word: count / num_of_tokens for (word, count) in counts.items()})
    else:
            
        default_model = defaultdict(lambda :0.0) if not add_one else defaultdict(lambda :1.0/len(vocabulary))
        model = defaultdict(lambda : default_model)
        for gram, next_character_counts in counts.items():
            n = np.sum(list(next_character_counts.values()))

            if add_one:
              model[gram] = defaultdict(lambda: 1.0/(n + len(vocabulary)), {word: (count +1.0) / (n + len(vocabulary)) for (word, count) in next_character_counts.items()})
            else:
              model[gram] = defaultdict(lambda : 0.0, {word: count / n for (word, count) in next_character_counts.items()})

    return model






def lm(n, vocabulary, data_file_path, add_one):
  # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
  # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
  # data_file_path - the data_file from which we record probabilities for our model
  # add_one - True/False (use add_one smoothing or not)

  # TODO
  tweets = read_tweets(data_file_path)
  return build_probs_model(n, vocabulary, tweets, add_one)

**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.

In [12]:
def calc_perplexity(n, model1, model2):
    entropy_sum = 0.0
    probs_n = 0.0
    if n == 1:
        for token, prob in model1.items():
            entropy_sum += prob * np.log2(model2[token] if token in model2 else 0.0)
            probs_n += 1
    else:
        for prefix_tokens, next_probs in model1.items():
            for token, prob in next_probs.items():
                other_prob = model2[prefix_tokens][token] 
                entropy_sum += prob * np.log2(other_prob if other_prob > 0 else 1)
                probs_n += 1

    entropy = -1/probs_n * entropy_sum

    return np.power(2, entropy)

def eval(n, model, data_file):
  # n - the n-gram that you used to build your model (must be the same number)
  # model - the dictionary (model) to use for calculating perplexity
  # data_file - the tweets file that you wish to claculate a perplexity score for

  # TODO

  other_model = lm(n, vocabulary, data_file, False)
  return calc_perplexity(other_model, model)


**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [13]:
def match(n, add_one):
  # n - the n-gram to use for creating n-gram models
  # add_one - use add_one smoothing or not

  vocabulary = preprocess()

  language_to_model = {}
  for file_name in os.listdir(folder):
    language = os.path.splitext(file_name)[0]
    if language in language_to_model or 'test' in language:
      continue
    
   
    file_path = os.path.join(folder, file_name)

    model = lm(n, vocabulary, file_path, add_one)
    language_to_model[language] = model

  languages = sorted(list(language_to_model.keys()))

  rows = []
  for index, language1 in enumerate(languages):
    current_res = []
    for index, language2 in enumerate(languages):
      perplexity = calc_perplexity(n, language_to_model[language1], language_to_model[language2])

      current_res.append(perplexity)
    
    rows.append(current_res)
  
  return pd.DataFrame(rows, columns =languages, index=languages)


match(2, True) 

,en,es,fr,in,it,nl,pt,tl
en,1.026406,1.029796,1.028929,1.029299,1.029722,1.028642,1.030553,1.029046
es,1.005705,1.005453,1.005657,1.005747,1.005661,1.005721,1.005634,1.005717
fr,1.004910,1.004899,1.004776,1.004934,1.004886,1.004902,1.004909,1.004940
in,1.004494,1.004503,1.004499,1.004467,1.004503,1.004495,1.004512,1.004488
it,1.004356,1.004353,1.004349,1.004359,1.004334,1.004357,1.004357,1.004359
nl,1.004328,1.004334,1.004330,1.004331,1.004333,1.004318,1.004337,1.004333
pt,1.004314,1.004307,1.004310,1.004317,1.004310,1.004314,1.004294,1.004315
tl,1.004295,1.004298,1.004298,1.004294,1.004298,1.004297,1.004301,1.004288


**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [ ]:
for n in range(1,5):
  for add_one in [True, False]:
    print(match(n, add_one))

**Part 6**

Each line in the file test.csv contains a sentence and the language it belongs to. Write a function that uses your language models to classify the correct language of each sentence.

Important note regarding the grading of this section: this is an open question, where a different solution will yield different accuracy scores. any solution that is not trivial (e.g. returning 'en' in all cases) will be excepted. We do reserve the right to give bonus points to exceptionally good/creative solutions.

In [48]:
def build_languages_models(n, add_one):

  vocabulary = preprocess()

  language_to_model = {}
  for file_name in os.listdir(folder):
    language = os.path.splitext(file_name)[0]
    if language in language_to_model or 'test' in language:
      continue
      
    
    file_path = os.path.join(folder, file_name)

    model = lm(n, vocabulary, file_path, add_one)
    language_to_model[language] = model
  return language_to_model

language_to_model = build_languages_models(2, False)

In [19]:
TEST_FILE = 'test.csv'
test_df = pd.read_csv(os.path.join(folder, TEST_FILE))[['tweet_text', 'label']]
test_df["tweet_text"] = test_df["tweet_text"].str.lower()

In [75]:
def classify_language(tweet, n):
  
  best_language_prob = float('-inf')
  best_language = None

  for language, model in language_to_model.items():
    index = 0

    language_log_prob = 0.0
    while index + n < len(tweet):
      if n == 1:
        token = tweet[index: index + 1]
        prob = model[token]
        prob = prob if prob > 0 else 1
        language_log_prob += np.log(prob)
      else:
        tokens = tweet[index: index + n]
        prev = tokens[0:-1]
        next = tokens[-1]

        prob = model[prev][next]
        prob = prob if prob > 0 else 1
        language_log_prob += np.log(prob)
      index += 1
    
    if language_log_prob > best_language_prob:
      best_language_prob = language_log_prob
      best_language = language
    

  return best_language


# #type(language_to_model['en'])

# ind = 1
# xx = classify_language(test_df.iloc[ind]['tweet_text'], 2)
# xx, test_df.iloc[ind]['label']

# #test_df["predicted_label"] = test_df["tweet_text"].apply(lambda t: classify_language(t, 1))


In [67]:
test_df["predicted_label"] = test_df["tweet_text"].apply(lambda t: classify_language(t, 2))

In [68]:
len(test_df[test_df['label'] == test_df['predicted_label']])/len(test_df)

0.7709713714214277

In [69]:
#test_df[test_df['label'] != test_df['predicted_label']].sample(2)
# test_df[test_df['predicted_label'].isnull()]

,tweet_text,label,predicted_label


**Part 7**

Calculate the F1 score of your output from part 6. (hint: you can use https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html). 


In [74]:
from sklearn.metrics import f1_score

f1_score(test_df['label'].tolist(), test_df['predicted_label'].tolist(), average='micro')
#test_df['predicted_label'].tolist()

0.7709713714214276

# **Good luck!**